### Pipeline de RAG (Retrieval-Augmented Generation)
Este notebook implementa um fluxo completo de **extração, processamento e indexação de documentos** para um sistema de RAG. O código foi reorganizado e documentado para melhor entendimento.

### Estrutura:
- Extração de texto de PDFs
- Limpeza e divisão em chunks
- Criação de embeddings (OpenAI)
- Indexação com FAISS
- Consultas e avaliação

### 1. Importação de bibliotecas e configuração

In [31]:
import os
import re
import json
import pickle
import unidecode
import numpy as np
import pdfplumber
import faiss
from dotenv import load_dotenv
from openai import OpenAI

# Carregar variáveis de ambiente
load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

### 2. Extração de texto dos PDFs
Aqui utilizamos a biblioteca `pdfplumber` para abrir os PDFs e extrair o texto bruto.

In [32]:
pdf_folder = r"D:\\Biopark\\4p\\PI3\\documents\\raw"

# Função de limpeza básica do texto
def clean_text(text):
    if not text:
        return ""
    text = unidecode.unidecode(text)  # remover acentos
    text = re.sub(r'\s+', ' ', text)  # normalizar espaços
    return text.strip()

# Carregar e limpar textos
pdf_texts = []
for file_name in os.listdir(pdf_folder):
    if file_name.endswith(".pdf"):
        with pdfplumber.open(os.path.join(pdf_folder, file_name)) as pdf:
            text = " ".join([page.extract_text() for page in pdf.pages if page.extract_text()])
            pdf_texts.append(clean_text(text))

print(f"Total de documentos processados: {len(pdf_texts)}")

Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P2' is an invalid float value
Cannot set gray non-stroke color because /'P3' is an invalid float value


Total de documentos processados: 10


### 3. Criação de Chunks
Dividimos os textos em pedaços menores (chunks) para facilitar a geração de embeddings e busca.

In [33]:
def chunk_text(text, max_tokens=500):
    words = text.split()
    for i in range(0, len(words), max_tokens):
        yield " ".join(words[i:i+max_tokens])

chunks = []
for i, doc in enumerate(pdf_texts):
    for chunk in chunk_text(doc):
        chunks.append({"doc_id": i, "text": chunk})

# Salvar em JSON
output_folder = r"D:\\Biopark\\4p\\PI3\\documents\\processed"
os.makedirs(output_folder, exist_ok=True)
output_file = os.path.join(output_folder, "chunks.json")

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

print(f"Total de chunks gerados: {len(chunks)}")

Total de chunks gerados: 80


### 4. Geração de Embeddings
Usamos a API da OpenAI para transformar os chunks em vetores numéricos (embeddings).

In [34]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

embeddings_list = []

for item in chunks:
    resp = client.embeddings.create(
        model="text-embedding-3-small",
        input=item["text"]
    )
    embedding = resp.data[0].embedding
    embeddings_list.append({"doc_id": item["doc_id"], "text": item["text"], "embedding": embedding})

print(f"Embeddings gerados: {len(embeddings_list)}")

Embeddings gerados: 80


### 5. Indexação com FAISS
Armazenamos os embeddings em um índice vetorial usando **FAISS** para consultas rápidas.

In [35]:
# Criar matriz de embeddings
X = np.array([item["embedding"] for item in embeddings_list]).astype("float32")

# Criar índice FAISS
index = faiss.IndexFlatL2(X.shape[1])
index.add(X)

# Salvar índice
with open(os.path.join(output_folder, "faiss_index.pkl"), "wb") as f:
    pickle.dump((index, embeddings_list), f)

print(f"Total de vetores indexados: {index.ntotal}")

Total de vetores indexados: 80


### 6. Consultas no Índice
Permite buscar documentos relevantes a partir de uma pergunta do usuário.

In [36]:
def gerar_embedding(texto):
    resp = client.embeddings.create(
        model="text-embedding-3-small",
        input=texto
    )
    return np.array(resp.data[0].embedding, dtype="float32")

# Exemplo de busca
consulta = "Qual contrato tem a empresa 3G INOVACAO - CONSULTORIA EMPRESARIAL LTDA?"
vec = gerar_embedding(consulta).reshape(1, -1)

distancias, indices = index.search(vec, k=3)

for i, idx in enumerate(indices[0]):
    print(f"Resultado {i+1} - Distância: {distancias[0][i]:.4f}")
    print(embeddings_list[idx]["text"][:200], "...\n")

Resultado 1 - Distância: 0.9592
CONTRATO DE ASSOCIACAO DE EMPRESA Pelo presente instrumento e na melhor forma de direito, de um lado, 3G INOVACAO - CONSULTORIA EMPRESARIAL LTDA, pessoa juridica de direito privado, inscrita no CNPJ s ...

Resultado 2 - Distância: 0.9997
em parte, por qualquer razao que seja, as demais continuarao em pleno vigor, a menos que o objeto deste Contrato seja afetado. Paragrafo Segundo: Este contrato representa a totalidade do acordo e ente ...

Resultado 3 - Distância: 1.0083
da empresa no Programa de Empresa Associada, restando encerrada a partir do momento em que o presente contrato for extinto, nao configurando locacao. D4Sign a9abc068-c8be-400e-85b3-3f4b53706107 - Para ...



In [37]:
## Consultas v2

def gerar_embedding(texto):
    resp = client.embeddings.create(
        model="text-embedding-3-small",
        input=texto
    )
    return np.array(resp.data[0].embedding, dtype="float32")

consultas = ["Qual contrato tem a empresa 3G INOVACAO - CONSULTORIA EMPRESARIAL LTDA?",
             "Qual contrato contem THAIS BIANCA BOGADO YARED"]
# Exemplo de busca
for con in consultas:
    print(f"\n\n{con}\n")
    vec = gerar_embedding(con).reshape(1, -1)
    distancias, indices = index.search(vec, k=3)

    for i, idx in enumerate(indices[0]):
        print(f"Resultado {i+1} - Distância: {distancias[0][i]:.4f}")
        print(embeddings_list[idx]["text"][:200], "...\n")



Qual contrato tem a empresa 3G INOVACAO - CONSULTORIA EMPRESARIAL LTDA?

Resultado 1 - Distância: 0.9596
CONTRATO DE ASSOCIACAO DE EMPRESA Pelo presente instrumento e na melhor forma de direito, de um lado, 3G INOVACAO - CONSULTORIA EMPRESARIAL LTDA, pessoa juridica de direito privado, inscrita no CNPJ s ...

Resultado 2 - Distância: 1.0001
em parte, por qualquer razao que seja, as demais continuarao em pleno vigor, a menos que o objeto deste Contrato seja afetado. Paragrafo Segundo: Este contrato representa a totalidade do acordo e ente ...

Resultado 3 - Distância: 1.0089
deste contrato nao constituira novacao ou renuncia, nem afetara o direito da Parte prejudicada de exigir seu cumprimento a qualquer tempo. Paragrafo Primeiro: Caso qualquer uma das clausulas deste con ...



Qual contrato contem THAIS BIANCA BOGADO YARED

Resultado 1 - Distância: 1.0992
caso a assinatura do presente instrumento seja feita em formato eletronico, estara reconhecida a veracidade, autenticidade, integ